<a href="https://colab.research.google.com/github/KonradGonrad/PyTorch-deep-learning/blob/main/extras%20/exercises/04_pytorch_custom_datasets_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 04. PyTorch Custom Datasets Exercises Template

Welcome to the 04. PyTorch Custom Datasets exercise template.

The best way to practice PyTorch code is to write more PyTorch code.

So read the original notebook and try to complete the exercises by writing code where it's required.

Feel free to reference the original resources whenever you need but should practice writing all of the code yourself.

## Resources

1. These exercises/solutions are based on [notebook 04 of the Learn PyTorch for Deep Learning course](https://www.learnpytorch.io/04_pytorch_custom_datasets/).
2. See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/vsFMF9wqWx0).
3. See [other solutions on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/extras/solutions).

In [ ]:
# Check for GPU
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
# Import torch
import torch
from torch import nn

# Exercises require PyTorch > 1.10.0
print(torch.__version__)

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

## 1. Our models are underperforming (not fitting the data well). What are 3 methods for preventing underfitting? Write them down and explain each with a sentence.

## 2. Recreate the data loading functions we built in [sections 1, 2, 3 and 4 of notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/). You should have train and test `DataLoader`'s ready to use.

In [ ]:
# 1. Get data
import requests
from pathlib import Path
import zipfile

DATA_PATH = Path("data/")
IMAGES_PATH = DATA_PATH / "pizza_steak_sushi"

if IMAGES_PATH.is_dir():
  print(f"{IMAGES_PATH} already exist.")
else:
  IMAGES_PATH.mkdir(parents=True, exist_ok = True)

  request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")

  with open(DATA_PATH / "pizza_steak_sushi.zip", 'wb') as f:
    f.write(request.content)

  with zipfile.ZipFile(DATA_PATH / "pizza_steak_sushi.zip", 'r') as zipf:
    zipf.extractall(IMAGES_PATH)

In [ ]:
# 2. Become one with the data
import os

def walk_through_dir(dir_path):
  """Walks through dir_path returning file counts of its contents."""
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

walk_through_dir(IMAGES_PATH)

In [ ]:
# Setup train and testing paths
TRAIN_PATH = IMAGES_PATH / "train"
TEST_PATH = IMAGES_PATH / "test"

In [ ]:
# Visualize an image
import random
from PIL import Image
import matplotlib.pyplot as plt

ALL_IMAGES_PATHS = list(IMAGES_PATH.glob("*/*/*.jpg"))

random_image = random.choice(ALL_IMAGES_PATHS)
random_image_label = random_image.parent.stem

random_img = Image.open(random_image)

In [ ]:
# Do the image visualization with matplotlib
plt.figure(figsize=(10,7))
plt.imshow(random_img)
plt.title(random_image_label)
plt.axis("off");

We've got some images in our folders.

Now we need to make them compatible with PyTorch by:
1. Transform the data into tensors.
2. Turn the tensor data into a `torch.utils.data.Dataset` and later a `torch.utils.data.DataLoader`.

In [ ]:
# 3.1 Transforming data with torchvision.transforms
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

data_transform = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
])


In [ ]:

# Write transform for turning images into tensors


In [ ]:
# Write a function to plot transformed images
import torchvision
import pathlib

def plot_transformed_images(image_paths: pathlib.Path,
                            transform = torchvision.transforms,
                            n: int = 3,
                            seed: torch.seed = 42
                            ):
  random.seed(seed)
  random_image_paths = random.sample(image_paths, k=n)
  for image_path in random_image_paths:
    with Image.open(image_path) as f:
      fig, ax = plt.subplots(1, 2)
      ax[0].set_title(f"Original shape: {f.size}")
      ax[0].axis("off")
      ax[0].imshow(f)

      f_transformed = transform(f)
      ax[1].set_title(f"New shape: {f_transformed.shape}")
      ax[1].axis("off")
      ax[1].imshow(f_transformed.permute(1, 2, 0))

plot_transformed_images(image_paths = ALL_IMAGES_PATHS,
                        transform = data_transform,
                        n = 3,
                        seed = 42)

### Load image data using `ImageFolder`

In [ ]:
# Use ImageFolder to create dataset(s)
from torchvision.datasets import ImageFolder

TRAIN_DIR = IMAGES_PATH / 'train'
TEST_DIR = IMAGES_PATH / 'test'

TRAIN_TRANSFORM = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.ToTensor()
])

TEST_TRANSFORM = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.ToTensor()
])

train_dataset = ImageFolder(root=TRAIN_DIR,
                            transform=TRAIN_TRANSFORM,
                            target_transform=None)

test_dataset = ImageFolder(root=TEST_DIR,
                           transform=TEST_TRANSFORM,
                           target_transform=None)

print(f"Train data:\n{train_dataset}\nTest data:\n{test_dataset}")

In [ ]:
# Get class names as a list
class_names = train_dataset.classes
class_names

In [ ]:
# Can also get class names as a dict
class_dict = train_dataset.class_to_idx
class_dict

In [ ]:
# Check the lengths of each dataset
len(train_dataset), len(test_dataset)

In [ ]:
# Turn train and test Datasets into DataLoaders
from torch.utils.data import DataLoader
import os

NUM_WORKERS = os.cpu_count()
BATCH_SIZE = 1

train_dataloader = DataLoader(train_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              num_workers=NUM_WORKERS)

test_dataloader = DataLoader(test_dataset,
                             batch_size=BATCH_SIZE,
                             shuffle=False,
                             num_workers=NUM_WORKERS)

In [ ]:
# How many batches of images are in our data loaders?
len(train_dataloader)

## 3. Recreate `model_0` we built in section 7 of notebook 04.

In [ ]:
class TinyVGG(nn.Module):
  def __init__(self,
               input_channels: int,
               hidden_channels: int,
               output_channels: int):
    super().__init__()
    self.layer_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_channels,
                  out_channels=hidden_channels,
                  kernel_size = 3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_channels,
                  out_channels=hidden_channels,
                  kernel_size= 3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2,
                     padding=0)
        )
    self.layer_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_channels,
                  out_channels=hidden_channels,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_channels,
                  out_channels=hidden_channels,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2,
                     padding=0)
    )
    self.output_layer = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_channels*13*13,
                  out_features=output_channels)
    )

  def forward(self, x):
    x = self.layer_1(x)
    #print(x.shape)
    x = self.layer_2(x)
    #print(x.shape)
    x = self.output_layer(x)
    #print(x.shape)
    return x

## 4. Create training and testing functions for `model_0`.

In [ ]:
import torch

def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device):

  # Put the model in train mode
  model.train()

  # Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0

  # Loop through data loader and data batches
  for Batch, (X, y) in enumerate(dataloader):
    # Send data to target device
    X, y = X.to(device), y.to(device)
    # 1. Forward pass
    y_logits = model(X)
    # 2. Calculate and accumulate loss
    loss = loss_fn(y_logits, y)
    train_loss += loss.item()
    # 3. Optimizer zero grad
    optimizer.zero_grad()

    # 4. Loss backward
    loss.backward()

    # 5. Optimizer step
    optimizer.step()

    # Calculate and accumualte accuracy metric across all batches
    y_pred = torch.argmax(torch.softmax(y_logits, dim=1), dim=1)
    train_acc += (y_pred == y).sum().item()/len(y_pred)

  # Adjust metrics to get average loss and average accuracy per batch
  train_loss /= len(dataloader)
  train_acc /= len(dataloader)

  return train_loss, train_acc

In [ ]:
model_0 = TinyVGG(input_channels = 3,
                  hidden_channels=10,
                  output_channels=len(class_names))

for i in range(2):
  l, a = train_step(model=model_0,
            dataloader=train_dataloader,
            loss_fn=nn.CrossEntropyLoss(),
            optimizer = torch.optim.Adam(params = model_0.parameters(),
                                        lr = 0.01),
            device='cpu')
  print(l, a)

In [ ]:
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device
              ):

  # Put model in eval mode
  model.eval()

  # Setup the test loss and test accuracy values
  test_loss, test_acc = 0, 0

  # Turn on inference context manager
  with torch.inference_mode():
    # Loop through DataLoader batches
    for batch, (X, y) in enumerate(dataloader):
      # Send data to target device
      X, y = X.to(device), y.to(device)

      # 1. Forward pass
      y_logits = model(X)

      # 2. Calculuate and accumulate loss
      loss = loss_fn(y_logits, y)
      test_loss += loss.item()
      # Calculate and accumulate accuracy
      y_pred = torch.argmax(y_logits, dim=1)
      test_acc += ((y_pred == y).sum().item() / len(y_pred))

  # Adjust metrics to get average loss and accuracy per batch
  test_loss /= len(dataloader)
  test_acc /= len(dataloader)

  return test_loss, test_acc

In [ ]:
for i in range(2):
  l, a = test_step(model_0,
            dataloader=test_dataloader,
            loss_fn=nn.CrossEntropyLoss(),
            device='cpu')
  print(l, a)

In [ ]:
from tqdm.auto import tqdm

def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          device: torch.device,
          loss_fn: torch.nn.Module = nn.CrossEntropyLoss(),
          epochs: int = 5):

  # Create results dictionary
  results = {"train_loss": [],
             "train_acc": [],
             "test_loss": [],
             "test_acc": []}

  # Loop through the training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
    # Train step
    train_loss, train_acc = train_step(model=model,
                                       dataloader=train_dataloader,
                                       loss_fn=loss_fn,
                                       optimizer=optimizer,
                                       device=device)
    # Test step
    test_loss, test_acc = test_step(model=model,
                                    dataloader=test_dataloader,
                                    loss_fn=loss_fn,
                                    device=device
                                    )

    # Print out what's happening
    print(f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_acc: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_acc: {test_acc:.4f}"
    )

    # Update the results dictionary
    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)

  # Return the results dictionary
  return results


In [ ]:
import matplotlib.pyplot as plt

def plot_results(results: dict[str, list[float]]) -> plt.plot:
  train_loss, test_loss = results["train_loss"], results["test_loss"]
  train_acc, test_acc = results["train_acc"], results["test_acc"]

  epochs = range(len(results["train_loss"]))

  plt.figure(figsize=(16, 8))

  plt.subplot(1, 2, 1)
  plt.title('Loss')
  plt.plot(epochs, train_loss, label = 'train')
  plt.plot(epochs, test_loss, label = 'test')
  plt.legend()

  plt.subplot(1, 2, 2)
  plt.title('Accuracy')
  plt.plot(epochs, train_acc, label = 'train')
  plt.plot(epochs, test_acc, label = 'test')
  plt.legend()

  plt.show();


## 5. Try training the model you made in exercise 3 for 5, 20 and 50 epochs, what happens to the results?
* Use `torch.optim.Adam()` with a learning rate of 0.001 as the optimizer.

In [ ]:
# Fresh model
model_01 = TinyVGG(input_channels = 3,
                  hidden_channels=10,
                  output_channels=len(class_names))

# Train for 5 epochs
torch.manual_seed(42)
torch.cuda.manual_seed(42)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(#TODO,
                             params=model_01.parameters(),
                             lr=0.001)

EPOCHS = 5

model_0_5e = train(model=model_01,
                   train_dataloader=train_dataloader,
                   test_dataloader=test_dataloader,
                   optimizer=optimizer,
                   loss_fn=loss_fn,
                   device='cpu',
                   epochs=EPOCHS)

plot_results(model_0_5e)

In [ ]:
# Fresh model
model_02 = TinyVGG(input_channels = 3,
                  hidden_channels=10,
                  output_channels=len(class_names))

# Train for 20 epochs
torch.manual_seed(42)
torch.cuda.manual_seed(42)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(#TODO,
                             params=model_02.parameters(),
                             lr=0.001)

EPOCHS = 20

model_0_20e = train(model=model_02,
                    train_dataloader = train_dataloader,
                    test_dataloader = test_dataloader,
                    optimizer = optimizer,
                    loss_fn=loss_fn,
                    device='cpu',
                    epochs = EPOCHS)

plot_results(model_0_20e)

In [ ]:
# Fresh model
model_03 = TinyVGG(input_channels = 3,
                  hidden_channels=10,
                  output_channels=len(class_names))

# Train for 20 epochs
torch.manual_seed(42)
torch.cuda.manual_seed(42)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(#TODO,
                             params=model_03.parameters(),
                             lr=0.001)

EPOCHS = 50

model_0_50e = train(model=model_03,
                    train_dataloader = train_dataloader,
                    test_dataloader = test_dataloader,
                    optimizer = optimizer,
                    loss_fn=loss_fn,
                    device='cpu',
                    epochs = EPOCHS)

plot_results(model_0_50e)

It looks like our model is starting to overfit towards the end (performing far better on the training data than on the testing data).

In order to fix this, we'd have to introduce ways of preventing overfitting.

## 6. Double the number of hidden units in your model and train it for 20 epochs, what happens to the results?

In [ ]:
# Double the number of hidden units and train for 20 epochs
torch.manual_seed(42)
torch.cuda.manual_seed(42)


It looks like the model is still overfitting, even when changing the number of hidden units.

To fix this, we'd have to look at ways to prevent overfitting with our model.

## 7. Double the data you're using with your model from step 6 and train it for 20 epochs, what happens to the results?
* **Note:** You can use the [custom data creation notebook](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/04_custom_data_creation.ipynb) to scale up your Food101 dataset.
* You can also find the [already formatted double data (20% instead of 10% subset) dataset on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/data/pizza_steak_sushi_20_percent.zip), you will need to write download code like in exercise 2 to get it into this notebook.

In [ ]:
# Download 20% data for Pizza/Steak/Sushi from GitHub
import requests
import zipfile
from pathlib import Path

# Setup path to data folder
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi_20_percent"

# If the image folder doesn't exist, download it and prepare it...
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

# Download pizza, steak, sushi data
with open(data_path / "pizza_steak_sushi_20_percent.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi_20_percent.zip")
    print("Downloading pizza, steak, sushi 20% data...")
    f.write(request.content)

# Unzip pizza, steak, sushi data
with zipfile.ZipFile(data_path / "pizza_steak_sushi_20_percent.zip", "r") as zip_ref:
    print("Unzipping pizza, steak, sushi 20% data...")
    zip_ref.extractall(image_path)

In [ ]:
# See how many images we have
walk_through_dir(image_path)

Excellent, we now have double the training and testing images...

In [ ]:
# Create the train and test paths
train_data_20_percent_path = image_path / "train"
test_data_20_percent_path = image_path / "test"

train_data_20_percent_path, test_data_20_percent_path

In [ ]:
# Turn the 20 percent datapaths into Datasets and DataLoaders
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader

simple_transform = transforms.Compose([
  transforms.Resize((64, 64)),
  transforms.ToTensor()
])

# Create datasets


# Create dataloaders


In [ ]:
# Train a model with increased amount of data
torch.manual_seed(42)
torch.cuda.manual_seed(42)

## 8. Make a prediction on your own custom image of pizza/steak/sushi (you could even download one from the internet) with your trained model from exercise 7 and share your prediction.
* Does the model you trained in exercise 7 get it right?
* If not, what do you think you could do to improve it?